# Run Sandwich Attacks on Swaps from the Public Mempool

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import sys
import math

current_path = sys.path[0]
sys.path.append(
    current_path[: current_path.find("defi-measurement")]
    + "liquidity-distribution-history"
)

sys.path.append("..")


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


from datetime import datetime, timezone


# Set display options
pd.set_option('display.max_colwidth', None)  # Display entire cell content
pd.set_option('display.max_rows', 50)    # Display all rows
pd.set_option('display.max_columns', None) # Display all columns


from dotenv import load_dotenv
from pool_state import v3Pool
from sqlalchemy import create_engine
from tqdm import tqdm

load_dotenv(override=True)
# from experiments.random_permutations import load_pool

from experiments.preload_pool_cache import load_pool_from_blob
from experiments.calculate_simple_sandwich import get_data


from decimal import getcontext

getcontext().prec = 100  # Set the precision high enough for our purposes


# Read in the environment variables
postgres_uri_mp = os.environ["POSTGRESQL_URI_MP"]
postgres_uri_us = os.environ["POSTGRESQL_URI_US"]
azure_storage_uri = os.environ["AZURE_STORAGE_CONNECTION_STRING"]


## Get the Data

### Populate the swap data we have with the block number that the swap appeared in

In [9]:
# Keep only the swap in subset for now
df_single_sorted = df_single_sorted[df_single_sorted.transaction_type == 'V3_SWAP_EXACT_IN'].drop(columns=['transaction_type', 'recipient', 'amountOut', 'amountInMax', 'payerIsUser', 'transaction_hash'])

In [5]:
df_single_sorted = get_data()

df_single_sorted

amountIn  \
pool                                       block_number                                                                                  
0x11b815efb8f581194ae79006d24e0d814b7697f6 17663582.0                                                              1000000000000000000   
                                           17663705.0                                                               550000000000000000   
                                           17664008.0    57896044618658097711785492504343953926634992332820282019728792003956564819968   
                                           17664143.0                                                              5003888921383035880   
                                           17664166.0    57896044618658097711785492504343953926634992332820282019728792003956564819968   
...                                                                                                                                ...   
0x5b40a1d8aeab3f412cfedf52ba3e958c7e7ad02e 17669429.0                                                                         99925095   
0x5b4bb43e57c0b5a9b3b28329d1ced52d3da042ce 17842184.0                                                            521364438696929637610   
0x5d171a15d950c174d4fd9df9c8cc872a9972475e 17795552.0                                                                98176066774320521   
0xb7c096a81011f93c55d6848e037e0231cc4af923 17705204.0                                                             35000000000000000000   
0xb79fa1e4835b558bac7f7c560d69d8d833ff68a9 17744042.0                                                                10000000000000000   

                                                                 amountOutMin  \
pool                                       block_number                         
0x11b815efb8f581194ae79006d24e0d814b7697f6 17663582.0              1853593534   
                                           17663705.0              1014298222   
                                           17664008.0              4871971661   
                                           17664143.0              9296674466   
                                           17664166.0              7218919681   
...                                                                       ...   
0x5b40a1d8aeab3f412cfedf52ba3e958c7e7ad02e 17669429.0    95468241613365843638   
0x5b4bb43e57c0b5a9b3b28329d1ced52d3da042ce 17842184.0        9111013848341382   
0x5d171a15d950c174d4fd9df9c8cc872a9972475e 17795552.0       88871570819507156   
0xb7c096a81011f93c55d6848e037e0231cc4af923 17705204.0                 7006058   
0xb79fa1e4835b558bac7f7c560d69d8d833ff68a9 17744042.0     6331905618869727468   

                                                                                             token0  \
pool                                       block_number                                               
0x11b815efb8f581194ae79006d24e0d814b7697f6 17663582.0    0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2   
                                           17663705.0    0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2   
                                           17664008.0    0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2   
                                           17664143.0    0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2   
                                           17664166.0    0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2   
...                                                                                             ...   
0x5b40a1d8aeab3f412cfedf52ba3e958c7e7ad02e 17669429.0    0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48   
0x5b4bb43e57c0b5a9b3b28329d1ced52d3da042ce 17842184.0    0x51fe2e572e97bfeb1d719809d743ec2675924edc   
0x5d171a15d950c174d4fd9df9c8cc872a9972475e 17795552.0    0x2c5bcad9ade17428874855913def0a02d8be2324   
0xb7c096a81011f93c55d6848e037e0231cc4af923 17705204.0    0x469a1e910faa202470238ea9d614ae16c3e8b0ee   
0xb79fa1e4835b558bac7f7c560d69d8d833ff68a

## Get a pool and do some initial testing

In [ ]:
# Given 17 seconds per sandwich, how many sandwiches can you make in a night?
hours_per_night = 8
seconds_per_sandwich = 17
cpus = 1

hours_per_night * 60 * 60 / seconds_per_sandwich * cpus

1694.1176470588234